# Line Visual Function Creation

This plot aims to generate a line graph which plots in the style of USAID/DevTech. 

In [1]:
import pandas as pd 
from os.path import join
from pyADVISE import visualize
from bokeh.io import show
from bokeh.models import ColumnDataSource, Div, HoverTool, LabelSet, Label, Legend, LegendItem

/home/lightner/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/lightner/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


ModuleNotFoundError: No module named 'pyADVISE'

In [2]:
# access sample dataset 
data = pd.read_csv(join('pyADVISE', 'data', 'practice.csv'), sep='|', low_memory=False)
data = data.drop('Unnamed: 0', axis=1)
#data_long = pd.wide_to_long(data, ["Var"], i=['countryname', 'countrycode', 'year'], j="Variable Code").reset_index()

In [3]:
data.describe()

,year,Variable Code
count,957180.000000,957180.000000
mean,2003.991240,108.981395
std,7.784192,62.913777
min,1991.000000,0.000000
25%,1997.000000,55.000000
50%,2004.000000,109.000000
75%,2011.000000,164.000000
max,2017.000000,217.000000


In [6]:
show(visualize.gen_line(data, country_list=['Algeria', 'Malawi', 'China'], var_list=['8'], var_labels=['GDP Per Capita'],
              line_width=8, title_text = 'Wohoo!'))

Loading BokehJS ...

In [7]:
# define the select data function for the ESDB database style 
def select_data(data, countries=('countryname', ['Malawi', 'China']), years=('year', (1995,2015)), variables=['3']): 

                # countries 
    data = data[data[countries[0]].isin(countries[1]) 
                # years
                & data[years[0]].between(years[1][0], years[1][1])   
                # variables
                & data['Variable Code'].isin(variables)
               ]
    
    return data


In [10]:
# Define USAID_style
def USAID_style(p): 
    palette = {'USAID Blue': '#002F6C', 'USAID Red': '#BA0C2F', 'Rich Black': '#212721', 'Medium Blue': '#0067B9',
    'Light Blue': '#A7C6ED', 'Dark Red': '#651D32', 'Dark Gray': '#6C6463', 'Medium Gray': '#8C8985', 'Light Gray': '#CFCDC9'}
    
    
    #p.title.text = title_name
    p.title.text_font_size='20pt'
    p.title.text_font = 'Gill Sans MT'
    p.legend.label_text_font = 'Gill Sans MT'
    
    p.legend.background_fill_color = palette['Light Gray']
    
    p.xgrid.visible = False
    p.title.text_color = palette['Dark Gray']
    p.title.text_font = 'Gill Sans MT'
    #p.title.text_font_style = "normal"
    p.grid.grid_line_color=palette['Dark Gray']
    p.grid.grid_line_width=2.0
    #p.xaxis.axis_label=''
    p.axis.axis_label_text_font = 'Gill Sans MT'
    p.axis.axis_label_text_color = palette['Dark Gray']
    p.axis.axis_label_text_font_style = 'normal'
    p.xaxis.minor_tick_line_color = None
    p.yaxis.minor_tick_line_color = None
    p.axis.axis_line_color= palette['Dark Gray']
    p.axis.axis_line_alpha= 0.5
    p.axis.axis_line_width= 2
    #p.axis.major_tick_line_color=None
    p.axis.minor_tick_line_color=None
    p.axis.major_label_text_font_size = '16pt'
    p.axis.major_label_text_color=palette['Dark Gray']
    p.axis.major_label_text_font='Gill Sans MT'
    p.outline_line_color = None
    
    #### Legend specifications
    p.legend.label_text_font_size='20pt'
    
    
    # grid line specifications 
    p.grid.grid_line_dash = 'dashed'
    p.grid.grid_line_alpha = 0.4
    
    #p.xaxis.axis_label_text_font_color = 'white'

    return p 

In [13]:
########################################################
####  This generages the pyADVISE visualize package
########################################################

# import packages
import os
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


from bokeh.plotting import figure
from bokeh.layouts import layout, widgetbox
from bokeh.models import ColumnDataSource, Div, HoverTool, LabelSet, LegendItem, Legend
from bokeh.models.widgets import Slider, Select, TextInput
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.models import Range1d
from bokeh.io import show, output_notebook
from bokeh.models import LogColorMapper
from bokeh.palettes import Viridis6 as palette
from bokeh.tile_providers import CARTODBPOSITRON

import ast
from pyproj import Proj, transform
import pyproj
from scipy.interpolate import spline


def gen_line(data, country_list=['Malawi', 'China'], var_list=['8'], var_labels=['GNI Per Capita'], year_tuple=(2008,2018), 
             y_axis='Y-Axis', x_axis='X-Axis', plot_width = 700, plot_height = 500, 
             title='Line Plot', country_var='countryname', year_var = 'year', variable_var='Variable Code', 
             smooth_line = False, line_width = 5, title_text = 'Line Graph', 
             line_pattern=['solid', 'dashed', 'solid', 'dashed'], dots=True):
    
    '''This function takes multiple countries, with one indicator variable and compares them accordingly. Or one country and 
    multiple indicators. At the moment, the function does not support multiple selections of both countries and indicators. '''
    
    output_notebook()
    
    # colors 
    palette = {'USAID Blue': '#002F6C', 'USAID Red': '#BA0C2F', 'Rich Black': '#212721', 'Medium Blue': '#0067B9',
    'Light Blue': '#A7C6ED', 'Dark Red': '#651D32', 'Dark Gray': '#6C6463', 'Medium Gray': '#8C8985', 'Light Gray': '#CFCDC9'}
    
    palette_names = list(palette.keys())
    
    
    line_style = line_pattern*(len(country_list)*len(var_list))
    
    #############################################
    ####  More than one country 
    #############################################
    
    
    if len(country_list)>=1 & len(var_list)==1: 
        data = select_data(data, countries=(country_var, country_list), years=(year_var, year_tuple), variables=var_list)

        d = {}
        for i in country_list: 
            small = data[data[country_var]==i]


            circles = {}
            lines = {}
            if smooth_line == False: 
                for v in var_list: 
                    lines[v] = [small['Var'].values, small[year_var].values]
                    circles[v] = [small['Var'].values, small[year_var].values]
            else: 
                print('Smooth options is still under construction.')
                values = small['Var'].values
                years = small[year_var].values
                # take code from ODI code

                # df['year'] = np.linspace(country_df['year'].min(), country_df['year'].max(), 180)
                # df[Var_Interest] = spline(country_df['year'], country_df[Var_Interest], df['year'])

                new_years = np.linspace(min(years), max(years), 180)

                # this is not working for some reason , not obvious.... says problem with interpolating floats...???
                new_values = spline(years, values, new_years)

                lines[v] = [new_values, new_years]
                circles[v] = [small['Var'].values, small[year_var].values]


            # return results 
            d[i] = [lines, circles]



        ##################################################
        #########  Generate Plot 
        ##################################################

        p = figure(plot_height=plot_height, plot_width=plot_width)


        plot_dictionary = {}
        source_dict = {}
        name_list = []
        labels = []
        n=0
        for i in country_list: 
            for f in range(0, len(var_list)): 


                if len(var_labels)>1:
                    name = i+': ' +var_labels[f]+'   '

                else: 
                    name = i+ '   '
                name_list.append(name)

                # generate the lines - noted by the [0] pointing to the lines dictionary 
                # this distinction is needed for the 
                s = p.line(x=d[i][0][var_list[f]][1], y=d[i][0][var_list[f]][0], 
                           color=palette[palette_names[n]], line_width=line_width, 
                           line_dash=line_style[n], line_dash_offset=4)


                if dots == True: 
                    source_dict[i+str(f)] = ColumnDataSource({'x': d[i][1][var_list[f]][1], 'y': d[i][1][var_list[f]][0], 'country': [i]*len(d[i][1][var_list[f]][1])})

                    r = plot_dictionary[i+str(f)] = p.circle('x', 'y', source = source_dict[i+str(f)], name=name, color=palette[palette_names[n]], size=(line_width*2))


                # legend outside the plot
                labels.append(LegendItem(label=dict(value=name), renderers=[s]))

                n+=1

        legend = Legend(items=labels, location=(0, 0), orientation='horizontal')
        p.add_layout(legend, 'above') 




        # Define tooltips

        Tip2 = '@'+'country'
        Tip1 = '@'+'x'
        Tip3 = '@'+'y'

        # define tooltips
        TT = [
            ('Country', Tip2),
             ('Year',   Tip1),
            ( 'Value', Tip3)
        ]

        # get list of renderers
        list_renderers = [plot_dictionary[i] for i in list(plot_dictionary.keys())]

        # add hover tool
        hover_circle = HoverTool(
            names = name_list,
            #renderers=list_renderers,
            tooltips=TT)

        p.add_tools(hover_circle)
    
    
    
    
    ##################################################
    ####### More than one variable 
    #################################################
    
    elif (len(country_list)==1) & (len(var_list)>=1): 
        
        
    
        data = select_data(data, countries=(country_var, country_list), years=(year_var, year_tuple), variables=var_list)

        d = {}
        for i in range(0, len(var_list)): 
            small = data[data[variable_var]==int(var_list[i])]


            if smooth_line == False: 
                lines = [small['Var'].values, small[year_var].values]
                circles = [small['Var'].values, small[year_var].values]
            else: 
                print('Smooth options is still under construction.')
                values = small['Var'].values
                years = small[year_var].values
                # take code from ODI code

                # df['year'] = np.linspace(country_df['year'].min(), country_df['year'].max(), 180)
                # df[Var_Interest] = spline(country_df['year'], country_df[Var_Interest], df['year'])

                new_years = np.linspace(min(years), max(years), 180)

                # this is not working for some reason , not obvious.... says problem with interpolating floats...???
                new_values = spline(years, values, new_years)

                lines[v] = [new_values, new_years]
                circles[v] = [small['Var'].values, small[year_var].values]


            # return results 
            d[var_labels[i]] = [lines, circles]


        ##################################################
        #########  Generate Plot 
        ##################################################

        p = figure(plot_height=500, plot_width=800)


        plot_dictionary = {}
        source_dict = {}
        name_list = []
        labels = []
        n=0
        
        
        for i in var_labels: 
            
            # add spacing between labels 
            name = i+ '   '
            name_list.append(name)

            # generate the lines - noted by the [0] pointing to the lines dictionary 
            # this distinction is needed for the 
            s = p.line(x=d[i][0][1], y=d[i][0][0], 
                       color=palette[palette_names[n]], line_width=line_width, 
                       line_dash=line_style[n], line_dash_offset=4)


            if dots == True: 
                
                source_dict[i] = ColumnDataSource({'x': d[i][1][1], 'y': d[i][1][0], 'variable': [i]*len(d[i][1][1])})

                r = plot_dictionary[i] = p.circle('x', 'y', source = source_dict[i], name=name, color=palette[palette_names[n]], size=(line_width*2))


            # legend outside the plot
            labels.append(LegendItem(label=dict(value=name), renderers=[s]))

            n+=1

        
        legend = Legend(items=labels[0:], location=(0, 0), orientation='horizontal')
        p.add_layout(legend, 'above') 
        
        #legend = Legend(items=[labels[1]], location=(0, 0), orientation='horizontal')
        #p.add_layout(legend, 'above')



        # Define tooltips

        Tip2 = '@'+'variable'
        Tip1 = '@'+'x'
        Tip3 = '@'+'y'

        # define tooltips
        TT = [
            ('Variable', Tip2),
             ('Year',   Tip1),
            ( 'Value', Tip3)
        ]

        # get list of renderers
        list_renderers = [plot_dictionary[i] for i in list(plot_dictionary.keys())]

        # add hover tool
        hover_circle = HoverTool(
            names = name_list,
            #renderers=list_renderers,
            tooltips=TT)

        p.add_tools(hover_circle)
            
    
    
    
    
    
    
    
    p = USAID_style(p)
    p.title.text = 'Figure 3: Comparison of multiple countries' +"³"
    p.legend.background_fill_color = 'white'
    
    return p


In [186]:
show(gen_line(data, country_list=['Malawi'], var_list=['3', '8'], var_labels=['GDP Per Capita', 'FDI (Nominal)'] ,line_width=8))

Loading BokehJS ...

{'GDP Per Capita': [[array(['2.3594326102585352', '2.409950945135769', '2.4908825233058254',
       '2.4328595935376365', '2.404181251114393', '2.3388132408792863'],
      dtype=object), array([1995, 1996, 1997, 1998, 1999, 2000])], [array(['2.3594326102585352', '2.409950945135769', '2.4908825233058254',
       '2.4328595935376365', '2.404181251114393', '2.3388132408792863'],
      dtype=object), array([1995, 1996, 1997, 1998, 1999, 2000])]], 'FDI (Nominal)': [[array(['343.214336840477', '401.6997120265073', '392.3639649160428',
       '443.76256354502027', '449.5054533405858', '459.0504959936701'],
      dtype=object), array([1995, 1996, 1997, 1998, 1999, 2000])], [array(['343.214336840477', '401.6997120265073', '392.3639649160428',
       '443.76256354502027', '449.5054533405858', '459.0504959936701'],
      dtype=object), array([1995, 1996, 1997, 1998, 1999, 2000])]]}
